# Dutch Corpora

### Instal tempo-embeddings

In [1]:
# Install tempo-embeddings from GitHub
# This can also refer to a specific version or branch
# %pip install ..

%pip install --upgrade pip  # Required for properly resolving dependencies
%pip uninstall -y tempo_embeddings  # Remove existing installation
%pip install --upgrade git+https://github.com/Semantics-of-Sustainability/tempo-embeddings.git


Processing /Users/jose/Repos/tempo-embeddings
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tempo_embeddings: filename=tempo_embeddings-0.0.1-py3-none-any.whl size=33929 sha256=a50c2e11448b8836b1df713af96e0c00262c5646c2852c4b4358daebecb80955
  Stored in directory: /Users/jose/Library/Caches/pip/wheels/de/25/96/d92b7a130b730e0ab67770d76841f36cb3d1f9cda32a4a539b
Successfully built tempo_embeddings
  Attempting uninstall: tempo_embeddings
    Found existing installation: tempo_embeddings 0.0.1
    Uninstalling tempo_embeddings-0.0.1:
      Successfully uninstalled tempo_embeddings-0.0.1
Note: you may need to restart the kernel to use updated packages.


### Import tempo-embeddings and start logging

In [2]:
# make sure installation has succeeded
import tempo_embeddings

from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

%load_ext autoreload

### Optional: Load NLP Pipeline

It is possible to use the Stanza library to pre-tokenize and split the text into sentences. This process is not perfect (it tends to overfragment the text if it has too many rare characters) and takes some extra amount of time and memory resources. If `nlp_pipeline = None` then a naive tokenization is done by splitting with spaces and stripping punctuation characters from the text.

In [3]:
%pip install stanza==1.7.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
nlp_pipeline = None

# import stanza
# stanza.download('nl')
# nlp_pipeline = stanza.Pipeline("nl", processors='tokenize')

## Load Data

The data needs to be downloaded and provided in the path configured in the next cell.

NOTE: You have to manually adapt the `DATA_DIR` below.

In [5]:
%autoreload now

import operator
from functools import reduce
from pathlib import Path
from tqdm import tqdm
from tempo_embeddings.text.corpus import Corpus

In [6]:
WINDOW_SIZE = 300 # Size of passages in characters
USE_FULL_SENTENCES = False # For now, this parameter overrides the window size. Instead of splitting arbitrarily the passages in WINDO_SIZE chunks, each Sentence will be a Passage

RANDOM_SAMPLE_ANP = 200
RANDOM_SAMPLE_STATEN_GENERAAL = 200

STATEN_GENERAAL_BLACKLIST = ["1987"]

FILTER_TERMS = ["duurzaam"]  # Search term(s) for filtering the corpus. If empty, it will only create Passages for the pieces of text that match the FILTER_TERMS

In [7]:
## NOTE: Adapt the `DATA_DIR` below manually!
## For a shared Google Drive, create a shortcut into your own Google Drive
## See https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab

try:
    import google.colab
    IN_COLAB = True
except ModuleNotFoundError:
    IN_COLAB = False

LOCAL_PATHS: list[Path] = [
    Path.home() / "Documents" / "SemanticsOfSustainability" / "data" / "Joris",
    Path.home() / "SEED_DATA" / "SemanticsSustainability", # local angel
    Path("/data/volume_2/data"),  # Research Cloud
    Path("/data/storage-semantics-of-sustainability/data/"), # New Research Cloud
    Path("/home/cschnober/data/"),  # Snellius
]

if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")

    DATA_DIR = Path("/content/drive/MyDrive/Data/")
else:
    try:
        DATA_DIR = next(path for path in LOCAL_PATHS if path.is_dir())
    except StopIteration as e:
        raise DirectoryNotFoundError(f"Data directory not found.") from e

assert DATA_DIR.is_dir(), f"Data dir '{DATA_DIR}' not found."

### ANP

In [8]:
ANP_DIR = DATA_DIR / "ANP"
assert RANDOM_SAMPLE_ANP == 0 or ANP_DIR.is_dir(), f"{ANP_DIR} not found."

In [9]:
import random


random.seed(0)

anp_files = list(ANP_DIR.glob("ANP_????.csv.gz"))

if RANDOM_SAMPLE_ANP and len(anp_files) > RANDOM_SAMPLE_ANP:
    anp_files = random.sample(
        list(ANP_DIR.glob("ANP_????.csv.gz")), k=RANDOM_SAMPLE_ANP
    )

print(f"Found {len(anp_files)} ANP Files")
anp_files[:10]

Found 2 ANP Files


[PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/ANP/ANP_1983.csv.gz'),
 PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/ANP/ANP_1984.csv.gz')]

In [10]:
anp_corpus = (
    reduce(
        operator.add,
        (
            Corpus.from_csv_file(
                path,
                filter_terms=FILTER_TERMS,
                text_columns=["content"],
                encoding="iso8859_15",
                compression="gzip",
                delimiter=";",
                window_size=WINDOW_SIZE,
                nlp_pipeline=nlp_pipeline
            )
            for path in tqdm(anp_files, unit="file")
        ),
    )
    if anp_files
    else Corpus()
)

len(anp_corpus)

100%|██████████| 2/2 [00:01<00:00,  1.75file/s]


54

### Staten Generaal

In [11]:
STATEN_GENERAAL_DIR = DATA_DIR / "StatenGeneraal"

assert RANDOM_SAMPLE_STATEN_GENERAAL == 0 or STATEN_GENERAAL_DIR.is_dir()

In [12]:
glob195x = "StatenGeneraal_19[0-9]?.csv.gz"  # Pattern for files from 1950-1999
glob20xx = "StatenGeneraal_2???.csv.gz"  # Pattern for files from 2000

files_195x = list(STATEN_GENERAAL_DIR.glob(glob195x))
files_20xx = list(STATEN_GENERAAL_DIR.glob(glob20xx))

sg_files = [
    file
    # Merge files from patterns
    for file in files_20xx + files_195x
    # Remove blacklisted files:
    for blacklisted in STATEN_GENERAAL_BLACKLIST
    if blacklisted not in file.name
]

if RANDOM_SAMPLE_STATEN_GENERAAL and RANDOM_SAMPLE_STATEN_GENERAAL < len(sg_files):
    sg_files = random.sample(sg_files, k=RANDOM_SAMPLE_STATEN_GENERAAL)

print(f"Found {len(sg_files)} STAATEN_G Files")
sorted(sg_files[:10])

Found 2 STAATEN_G Files


[PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/StatenGeneraal/StatenGeneraal_2017.csv.gz'),
 PosixPath('/Users/jose/SEED_DATA/SemanticsSustainability/StatenGeneraal/StatenGeneraal_2018.csv.gz')]

In [13]:
%autoreload now

import csv

csv.field_size_limit(100000000)

sg_corpus = (
    reduce(
        operator.add,
        (
            Corpus.from_csv_file(
                path,
                filter_terms=FILTER_TERMS,
                text_columns=["Content"],
                encoding="utf-8",
                compression="gzip",
                delimiter=";",
                window_size=WINDOW_SIZE,
                nlp_pipeline=nlp_pipeline
            )
            for path in tqdm(sg_files, unit="file")
        ),
    )
    if sg_files
    else Corpus()
)

len(sg_corpus)

100%|██████████| 2/2 [00:01<00:00,  1.16file/s]


1504

### Show the Passages that were created in the corpus

In [14]:
for p in sg_corpus.passages[:20]:
    print(len(p), p)

303 Passage('economische groei consequent milieuvernietiging op, in meerdere of mindere mate. Economische groei zou "duurzaam" te noemen zijn als die wordt gekoppeld aan technologische vooruitgang en toenemende efficiëntie. Zo zouden we de taart kunnen vergroten en alle milieuproblematiek de baas kunnen worden. Een', {'': '204', 'RecId': 'h-ek-20162017-14-3', 'chamber': 'EersteKamer', 'date': '2017-01-17', 'speakers': 'De voorzitter :|Mevrouw Teunissen ( PvdD ):|Mevrouw Vos ( GroenLinks ):|De heer Van Hattem ( PVV ):|Mevrouw Stienen ( D66 ):|De heer Verheijen ( PvdA ):|Mevrouw Meijer ( SP ):|De voorzitter :|Minister Schultz van Haegen-Maas Geesteranus :|Mevrouw Vos ( GroenLinks ):|Minister Schultz van Haegen-Maas Geesteranus :|Mevrouw Vos ( GroenLinks ):|Minister Schultz van Haegen-Maas Geesteranus :|De heer Verheijen ( PvdA ):|Minister Schultz van Haegen-Maas Geesteranus :|De heer Verheijen ( PvdA ):|Minister Schultz van Haegen-Maas Geesteranus :|De heer Van Hattem ( PVV ):|Minister S

### Merge

In [15]:
corpus = anp_corpus + sg_corpus
len(corpus)

1558

## Load Model

In [16]:
%autoreload now

LAYER = 9

from tempo_embeddings.embeddings.model import EmbeddingsMethod

from tempo_embeddings.embeddings.model import (
    RobertaModelWrapper,
    TransformerModelWrapper,
    XModModelWrapper,
    SentenceTransformerModelWrapper,
)

kwargs = {"accelerate": True}

### RoBERTa Models
# MODEL_NAME = "DTAI-KULeuven/robbertje-1-gb-non-shuffled"
# model_class = RobertaModelWrapper

### XMod Models
# MODEL_NAME = "facebook/xmod-base"
# kwargs["default_language"] = "nl_XX"
# model_class = XModModelWrapper

### BERT Models
# MODEL_NAME = "GroNLP/bert-base-dutch-cased"
# MODEL_NAME = "xlm-roberta-base"
# MODEL_NAME = "xlm-mlm-100-1280"
# model_class = TransformerModelWrapper

### Sentence Transformers
MODEL_NAME = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
# MODEL_NAME = "textgain/allnli-GroNLP-bert-base-dutch-cased"
model_class = SentenceTransformerModelWrapper

model = model_class.from_pretrained(MODEL_NAME, layer=LAYER, **kwargs)
model.embeddings_method = EmbeddingsMethod.MEAN
model

04:44:02 INFO:Created a temporary directory at /var/folders/79/zf67ls7520x9m4mj7nx6q07w0000gp/T/tmp5_u95kte
04:44:02 INFO:Writing /var/folders/79/zf67ls7520x9m4mj7nx6q07w0000gp/T/tmp5_u95kte/_remote_module_non_scriptable.py
/Users/jose/Repos/tempo-embeddings/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
04:44:05 WARNING:SentenceTransformerModelWrapper does not support different layers, ignoring.


## Create or Open existing Database

In [17]:
from tempo_embeddings.embeddings.vector_database import ChromaDatabaseManager

EMBED_CONFIG = {"type":"custom_model", "model": model}
BATCH_SIZE = 32

db = ChromaDatabaseManager(db_path="testing_db", embedder_name=MODEL_NAME,embedder_config=EMBED_CONFIG, batch_size=BATCH_SIZE)
db.connect()

collection_name = "anp_sg_corpus"

try:
    anp_sg_collection = db.create_new_collection(collection_name)
    created_new_collection = True
except ValueError:
    anp_sg_collection = db.get_existing_collection(collection_name)
    created_new_collection = False

04:44:07 INFO:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
04:44:07 INFO:Creating NEW Database in testing_db...
04:44:07 INFO:Created NEW collection 'anp_sg_corpus'


In [18]:
%autoreload now

## Compute & Save Embeddings (If DB didn't existed)

In [19]:
if created_new_collection:
    # Insert the passages from the corpus in the new collection
    db.insert_corpus(anp_sg_collection, corpus)
else:
    print("No new passages to insert")
    

Embeddings:  98%|█████████▊| 48/49 [00:16<00:00,  2.88batch/s]
04:44:23 INFO:Added 1558 new documents. Total = 1558


## TEST: Retrieve Records from Database

In [20]:
%autoreload now

# corpus = db.get_corpus(anp_sg_collection, filter_words=["toekomst"], where_obj={'$and': [{'year': {'$eq': '1983'}}, {'month': {'$eq': '7'}}]})
# corpus = db.get_corpus(anp_sg_collection, limit=100)
corpus = db.get_corpus(anp_sg_collection, filter_words=["duurzaam"])

for p in corpus.passages:
    print(p.text)

wist wie van u ik adresseerde . Ik zou toch déze minister willen vragen om enige reflectie op de term " duurzaamheid ". Ik vond het echt verheugend dat zij Martha Nussbaum citeerde en aanhaalde als inspiratiebron ; dat is goed nieuws . Ik zou daar dus toch graag een reflectie op krijgen . Daarna heb ik nog
dat bij alle bezwaren die zij zal ontmoeten in het daadwerkelijk inzetten op duurzaamheid en eerlijkheid in de richting van ontwikkelingslanden en in ons handelsbeleid , bezwaren op haar af zullen komen dat we daar economisch misschien wel eventjes last van hebben . Het vervelende is , we zijn volledig
langetermijndoelen op maatschappelijk verantwoord en duurzaam ondernemen worden vastgelegd . Ik vind dat duurzaamheid van het grootste belang is . Bedrijven vinden dat ook . De corporate - governancecode is daarop gericht . Ik vind dat het probleem dat we nu moeten oplossen niet de goede gelegenheid is om
en water . Voorzitter . De ChristenUnie zet zich in voor verbetering van de lucht

In [21]:
if len(corpus):
    db.create_new_collection("anp_duurzam", corpus)

Embeddings:  98%|█████████▊| 45/46 [00:15<00:00,  2.87batch/s]
04:44:39 INFO:Added 1464 new documents. Total = 1464
04:44:39 INFO:Created NEW collection 'anp_duurzam'
